### searching for the leaders of PoK

as the final step in our processing, we wanted to pull a small list of articles relevant to the leaders of PoK to round out the story of their structure. to do this, through other exploration we identified who the leaders were. in doing so we could begin to zero in on creating a narrative around the leaders

### import statements

once again using some nltk and pandas

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import os
from collections import Counter

import os
import chardet
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.tag import pos_tag
import re
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andyt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\andyt\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\andyt\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\andyt\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


### user defined functions

we borrow some functions again from previous code sets, just to help us establish a final rip of articles to find out about the leadership structure of the protectors of kronos

In [3]:
def search_for_terms(folder_path, terms, ignore_names):
    entities = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r") as file:
                full_text = file.read()
            tokenized_text = nltk.word_tokenize(full_text)
            tagged_text = nltk.pos_tag(tokenized_text)
            named_entities = nltk.ne_chunk(tagged_text)
            for entity in named_entities:
                if type(entity) == nltk.tree.Tree:
                    if entity.label() == 'PERSON':
                        name = ' '.join([word for word, tag in entity.leaves()])
                        if name not in ignore_names and terms[0] in full_text and name in full_text:
                            print(f"Found {terms[0]} and name '{name}' in document: {file_path}")
                            entities.append(name)
    return entities


def count_elements(list):
    count = Counter(list)
    for element, frequency in sorted(count.items(), key=lambda x: x[1], reverse=True):
        print(f"{element}: {frequency}")


def analyze_files_for_sentiment(folder_path, individuals_df, column_name):
    sia = SentimentIntensityAnalyzer()

    results = []

    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            # Read file contents
            with open(os.path.join(folder_path, file), 'rb') as f:
                raw_data = f.read()
                encoding = chardet.detect(raw_data)['encoding']
                text = raw_data.decode(encoding)

            tokens = word_tokenize(text)
            tokens = [t for t in tokens if t.lower() not in stopwords.words('english')]

            for item in individuals_df[column_name]:
                sentiment = 0
                count = 0
                keywords = set()

                for token in tokens:
                    if item.lower() in token.lower():
                        sentences = sent_tokenize(text)
                        for sentence in sentences:
                            if token in sentence:
                                tagged_sentence = pos_tag(word_tokenize(sentence))

                                for word, tag in tagged_sentence:
                                    if tag.startswith('N') and wordnet.synsets(word):
                                        keywords.add(word)

                                sentiment += sia.polarity_scores(sentence)['compound']
                                count += 1

                if count > 0:
                    results.append((file, item, round(sentiment/count, 2), list(keywords)))

    df = pd.DataFrame(results, columns=['File Name', 'Entity', 'Sentiment Score', 'Keywords'])

    return df


def combine_names(df):
    df['Name'] = df['FirstName'] + ' ' + df['LastName']
    return df


stop_words = ['Abila', 'Tethys', 'Kronos', 'Online', 'Currently', 'Tethyn', 'Jr', 'Ngohebo', 'Haneson Ngohebo' 'Carmen', 'Adrien Carmen']

### scanning for leader names

we're using the same code from part one to sift through all of the articles and print out each piece of information related to the leadership structure. it may be excessive to print each finding, but its also nice to just click on things as they pop up and check out the contents

In [5]:
try: 
    folder_path = "C:/Users/Andy/PycharmProjects/pythonProject/The_Last_Stand/TP-1_Kronos/articles"
    lead = search_for_terms(folder_path, ["leader", "name"], stop_words)    
except: 
    folder_path = "C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles"
    lead = search_for_terms(folder_path, ["leader", "name"], stop_words)

Found leader and name 'Daily' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\101.txt
Found leader and name 'Silvia Marek' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\101.txt
Found leader and name 'Histories' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\101.txt
Found leader and name 'Signs' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\132.txt
Found leader and name 'Juliana Vann' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\132.txt
Found leader and name 'Health Cesare Nespola' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\134.txt
Found leader and name 'Henk Bodrogi' in document: C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles\1

and again we are checking out who the hot commodities are

In [6]:
count_elements(lead)

Karel: 75
Elian Karel: 41
Henk Bodrogi: 18
Elian: 17
Sanjorge: 17
Juliana Vann: 15
Bodrogi: 14
Sten Sanjorge: 12
Silvia Marek: 11
Daily: 10
Michale Kraft: 10
Marek: 9
Reynolds: 8
Nespola: 7
Elodis: 7
Kraft: 7
Haneson Ngohebo: 7
Fluss: 5
Adrien Carman: 5
Rufus Drymiau: 5
Petrus Gerhard: 5
Mr. Kraft: 5
Araullo: 5
Worldwise: 5
Carman: 4
Sten: 4
Julian Vann: 4
Health Cesare Nespola: 3
Kapelou: 3
Marcella Trapani: 3
Miriam Avila: 3
Avila: 3
Antaura Karel: 3
Mestache Vardshvna: 3
Will: 3
Bodrodgi: 3
Joclyn Reynolds: 3
Mr. Fluss: 3
Pondo Reese: 3
Loblaw: 3
Danisliau: 3
Signs: 2
Pacific: 2
Jeroen: 2
Mr. Drymiau: 2
Athena: 2
Rather: 2
Vardshvna: 2
Petr Noriega: 2
Mr. Noriega: 2
Strikes Deal: 2
Sten Sanjorge Jr.: 2
Mr. Sanjorge: 2
Mr.: 2
Emeritus: 2
Ingrid Sanjorge: 2
Mr. Henk Bodrogi: 2
Jon L.: 2
Abila Post: 2
Sara Tuno: 2
Histories: 1
Turns Violent: 1
Beatriz Fluss: 1
Motto: 1
Tried Karel: 1
Traffic: 1
La Paz: 1
Mr. Sanjorge Jr: 1
Disappeared: 1
Sten Sanjorge Jr: 1
Willem: 1
John Rathburn: 1
C

### zeroing in on the leaders

after running through the articles and reading a few, we think we've identified who exactly the major players are in PoKs activism within Abila, so we create a small dataframe containing their names and run our sentiment analysis function to start to pull relevant articles for each leader, thus reducing the pool of articles to review

In [33]:
leaders_df = pd.DataFrame({'Name': ['Karel', 
                                    'Bodrogi', 
                                    'Osvaldo',
                                    'Marek',
                                    'Carmine'
                                    ]
                            })

display(leaders_df)

,Name
0,Karel
1,Bodrogi
2,Osvaldo
3,Marek
4,Carmine


In [34]:
try: 
    folder_path = "C:/Users/Andy/PycharmProjects/pythonProject/The_Last_Stand/TP-1_Kronos/articles"
    column_name = 'Name'
    sentiment_df = analyze_files_for_sentiment(folder_path, leaders_df, column_name)  
except: 
    folder_path = "C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/articles"
    column_name = 'Name'
    sentiment_df = analyze_files_for_sentiment(folder_path, leaders_df, column_name)

In [35]:
pd.set_option('display.max_columns', None)  # set to None to show all columns
pd.set_option('display.max_rows', None)  # set to None to show all rows
pd.set_option('display.max_colwidth', None)  # set to None to show all columns

display(sentiment_df.head())


,File Name,Entity,Sentiment Score,Keywords
0,0.txt,Karel,-0.13,"[death, toll, guards, ones, chiefs, chief, posters, recalls, images]"
1,101.txt,Marek,0.32,"[demands, citizens, employees, personnel, speech, people, responsibility, contributor, money, Silvia, government, vote, program, institute, leader, negligence]"
2,116.txt,Karel,-0.04,"[assassination, rap, ABILA, minutes, REMIND, Silvia, cases, ceramics, honor, resistance, LEADERs, homage, LEADER, worker, Wrap, MARTYRED, activist, Wednesday, June, candles, today, agriculture, company, guards, group, prison]"
3,116.txt,Bodrogi,0.00,"[pace, father, attention, founder]"
4,116.txt,Marek,0.34,"[rap, candles, today, shouts, Silvia, honor, moment]"


### final review

after doing some human scans of the articles, we determined some words that might help us zero in on the individuals associated with the leadership structure and identify key actions they performed as leaders of PoK. we finally apply some logic to zero in on the words that are most relevant to create a set of articles that will tell us the full story.

In [38]:
find_words = ['prison', 'drugs', 'demands', 'father', 'arrest', 'hardware']

filtered_df = sentiment_df[sentiment_df['Keywords'].apply(lambda x: any(word in x for word in find_words))]

filtered_df = filtered_df.reset_index()

display(filtered_df)

,index,File Name,Entity,Sentiment Score,Keywords
0,1,101.txt,Marek,0.32,"[demands, citizens, employees, personnel, speech, people, responsibility, contributor, money, Silvia, government, vote, program, institute, leader, negligence]"
1,2,116.txt,Karel,-0.04,"[assassination, rap, ABILA, minutes, REMIND, Silvia, cases, ceramics, honor, resistance, LEADERs, homage, LEADER, worker, Wrap, MARTYRED, activist, Wednesday, June, candles, today, agriculture, company, guards, group, prison]"
2,3,116.txt,Bodrogi,0.00,"[pace, father, attention, founder]"
3,6,132.txt,Karel,-0.84,"[anniversary, death, prison, Tuesday, leader]"
4,10,149.txt,Karel,-0.34,"[girl, time, stopping, clean, government, year, PRESIDENT, countryside, diseases, point, policy, Jr., Journal, reject, May, Chief, Friday, attention, crowd, concern, de, projector, terrorist, Abila, protest, speaking, jets, mark, offices, gathering, image, figure, World, voice, activist, road, Wednesday, demagog, event, death, front, Profile, company, scorn, guards, nation, island, martyrdom, group, chief, agenda, embers, prison, interview, pressure]"
5,11,154.txt,Bodrogi,-0.24,"[store, soybeans, father, life, tractor, making, town, fathers, purchases, hardware]"
6,12,154.txt,Osvaldo,-0.48,"[store, tractor, making, town, purchases, hardware]"
7,15,158.txt,Karel,-0.61,"[assassination, July, bone, ABILA, morning, report, task, government, MEMORY, nameless, LATER, year, matter, IN, force, cement, police, hours, servants, policy, DEATH, wounds, declaration, witness, body, cold, place, afterwards, instability, violence, wealth, A, Abila, search, cracks, rabble-rousers, transparency, cause, YEAR, town, research, Coroner, companies, sources, compatriots, future, charges, people, strength, Wrap, June, death, today, signs, guards, city, require, prison, recall]"
8,16,161.txt,Karel,-0.61,"[case, investigation, manner, ABILA, days, Requests, time, Illumination, TRAGICAL, Kraft, government, officials, LIFE, questions, accusations, force, CUT, assurances, police, information, custody, causes, lacerations, Chief, s, body, BACK, instability, violence, attorney, Abila, A, appearances, rabble-rousers, trauma, AT, family, Police, arrest, yesterday, cause, agreement, LOOK, court, age, cell, Questions, years, charges, people, linger, June, death, signs, blunt, trial, Homeland, city, SHORT, jail, release, demanding, abrasions, murder]"
9,22,18.txt,Bodrogi,0.00,"[father, founder, path]"


In [39]:
leader_articles_rip = sentiment_df.groupby('Entity').agg({'File Name': list, 'Sentiment Score': 'mean'})

display(leader_articles_rip)

,File Name,Sentiment Score
Entity,,
Bodrogi,"[116.txt, 134.txt, 154.txt, 18.txt, 183.txt, 187.txt, 205.txt, 211.txt, 217.txt, 237.txt, 255.txt, 264.txt, 317.txt, 35.txt, 391.txt, 399.txt, 4.txt, 408.txt, 447.txt, 448.txt, 459.txt, 465.txt, 502.txt, 51.txt, 531.txt, 572.txt, 622.txt, 628.txt, 647.txt, 662.txt, 69.txt, 696.txt, 698.txt, 735.txt, 741.txt, 743.txt, 814.txt, 839.txt]",-0.214737
Carmine,"[182.txt, 248.txt, 273.txt, 328.txt, 377.txt, 390.txt, 468.txt, 48.txt, 482.txt, 578.txt, 593.txt, 684.txt, 696.txt, 756.txt]",-0.340714
Karel,"[0.txt, 116.txt, 130.txt, 132.txt, 14.txt, 149.txt, 155.txt, 157.txt, 158.txt, 161.txt, 162.txt, 165.txt, 168.txt, 175.txt, 18.txt, 180.txt, 184.txt, 185.txt, 189.txt, 19.txt, 190.txt, 20.txt, 200.txt, 204.txt, 213.txt, 217.txt, 225.txt, 229.txt, 24.txt, 242.txt, 249.txt, 257.txt, 268.txt, 269.txt, 282.txt, 286.txt, 288.txt, 294.txt, 300.txt, 309.txt, 318.txt, 33.txt, 334.txt, 34.txt, 343.txt, 346.txt, 351.txt, 353.txt, 357.txt, 374.txt, 378.txt, 385.txt, 387.txt, 394.txt, 398.txt, 437.txt, 445.txt, 447.txt, 451.txt, 454.txt, 455.txt, 505.txt, 508.txt, 51.txt, 520.txt, 523.txt, 538.txt, 54.txt, 55.txt, 554.txt, 561.txt, 566.txt, 570.txt, 572.txt, 577.txt, 581.txt, 620.txt, 623.txt, 662.txt, 667.txt, 670.txt, 683.txt, 698.txt, 709.txt, 73.txt, 735.txt, 743.txt, 745.txt, 753.txt, 767.txt, 77.txt, 770.txt, 788.txt, 798.txt, 799.txt, 801.txt, 808.txt, 812.txt, 815.txt, 820.txt, ...]",-0.354135
Marek,"[101.txt, 116.txt, 135.txt, 18.txt, 207.txt, 239.txt, 358.txt, 414.txt, 454.txt, 508.txt, 570.txt, 572.txt, 581.txt, 620.txt, 662.txt, 670.txt, 698.txt, 767.txt, 773.txt, 840.txt, 843.txt]",0.189048
Osvaldo,"[154.txt, 248.txt, 317.txt, 448.txt, 468.txt, 482.txt, 531.txt, 612.txt, 69.txt, 740.txt]",-0.455000
